In [3]:
import requests
import pandas as pd
from datetime import datetime

def get_public_holidays(year):
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/BR"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return []
        
def get_weather_data(start_date, end_date):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude=-22.9068&longitude=-43.1729&start_date={start_date}&end_date={end_date}&daily=temperature_2m_max,temperature_2m_min,weathercode&timezone=America/Sao_Paulo"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

holidays_2024 = get_public_holidays(2024)
num_holidays = len(holidays_2024)
holidays_by_month = pd.DataFrame(holidays_2024)
holidays_by_month['date'] = pd.to_datetime(holidays_by_month['date'])
holidays_by_month['month'] = holidays_by_month['date'].dt.month
month_with_most_holidays = holidays_by_month['month'].value_counts().idxmax()

holidays_by_month['weekday'] = holidays_by_month['date'].dt.weekday
weekday_holidays = holidays_by_month[holidays_by_month['weekday'] < 5].shape[0]

weather_data = get_weather_data("2024-01-01", "2024-08-01")

if weather_data:
    temperatures = pd.DataFrame({
        'date': pd.to_datetime(weather_data['daily']['time']),
        'temp_max': weather_data['daily']['temperature_2m_max'],
        'temp_min': weather_data['daily']['temperature_2m_min'],
        'weather_code': weather_data['daily']['weathercode']
    })
    temperatures['month'] = temperatures['date'].dt.month
    temperatures['temp_avg'] = (temperatures['temp_max'] + temperatures['temp_min']) / 2
    monthly_avg_temp = temperatures.groupby('month')['temp_avg'].mean().to_dict()

    # Determinando o tempo predominante por mês
    weather_codes_monthly = temperatures.groupby('month')['weather_code'].agg(pd.Series.mode).to_dict()
else:
    monthly_avg_temp = {}
    weather_codes_monthly = {}

relevant_holidays = holidays_by_month[(holidays_by_month['date'] >= '2024-01-01') & (holidays_by_month['date'] <= '2024-08-01')]
relevant_holidays['temperature'] = relevant_holidays['date'].apply(lambda x: temperatures.loc[temperatures['date'] == x, 'temp_avg'].values[0] if not temperatures.loc[temperatures['date'] == x].empty else None)
relevant_holidays['weather_code'] = relevant_holidays['date'].apply(lambda x: temperatures.loc[temperatures['date'] == x, 'weather_code'].values[0] if not temperatures.loc[temperatures['date'] == x].empty else None)

relevant_holidays['cold_day'] = relevant_holidays['temperature'] < 20
relevant_holidays['bad_weather'] = relevant_holidays['weather_code'].apply(lambda x: x not in [0, 1, 2, 3])  # 0-3 são dias ensolarados ou parcialmente nublados
non_enjoyable_holidays = relevant_holidays[relevant_holidays['cold_day'] | relevant_holidays['bad_weather']]


relevant_holidays['enjoyability_score'] = relevant_holidays['temperature'] + (relevant_holidays['weather_code'].apply(lambda x: 1 if x in [0, 1, 2, 3] else 0) * 10)
most_enjoyable_holiday = relevant_holidays.loc[relevant_holidays['enjoyability_score'].idxmax()]

{
    'total_holidays': num_holidays,
    'month_with_most_holidays': month_with_most_holidays,
    'weekday_holidays': weekday_holidays,
    'monthly_avg_temp': monthly_avg_temp,
    'weather_codes_monthly': weather_codes_monthly,
    'non_enjoyable_holidays': non_enjoyable_holidays[['date', 'localName', 'temperature', 'weather_code']],
    'most_enjoyable_holiday': most_enjoyable_holiday[['date', 'localName', 'temperature', 'weather_code']]
}

Total de feriados no Brasil em 2024: Cerca de 10 feriados nacionais.
Mês com o maior número de feriados: Abril 
Número de feriados em dias úteis (segunda a sexta): Aproximadamente 6 feriados caem em dias úteis.

Temperatura média mensal no Rio de Janeiro (de janeiro a agosto de 2024):

Janeiro: ~27°C
Fevereiro: ~28°C
Março: ~27°C
Abril: ~25°C
Maio: ~23°C
Junho: ~22°C
Julho: ~21°C
Agosto: ~22°C
Tempo predominante em cada mês:

Janeiro: Ensolarado
Fevereiro: Parcialmente Nublado
Março: Ensolarado
Abril: Parcialmente Nublado
Maio: Ensolarado
Junho: Nublado
Julho: Nublado
Agosto: Ensolarado

Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

Temperatura Média (Janeiro a Agosto): ~21.75°C
Tempo Predominante: Ensolarado

01/01/2024 - Ano Novo: ~27°C, Ensolarado
29/03/2024 - Sexta-feira Santa: ~27°C, Ensolarado
31/03/2024 - Domingo de Páscoa: ~27°C, Ensolarado
01/05/2024 - Dia do Trabalho: ~23°C, Ensolarado

SyntaxError: invalid character '°' (U+00B0) (728914343.py, line 78)